![DeepNeuro](https://github.com/QTIM-Lab/DeepNeuro/raw/master/package_resources/logos/DeepNeuro_alt.PNG?raw=true)

# Running a DeepNeuro Module with Docker

In this notebook, you will learn how to run a DeepNeuro module from its Docker container. DeepNeuro modules are scripts created with DeepNeuro syntax, and their goal is usually to: load data from any state of preprocessing, run inference on a model trained to accomplish a certain task, and save out to the results to a location of your choosing. Today, we will be running the glioma segmentation module in DeepNeuro.

You will need to have Docker already installed for this tutorial. You can find some instructions on how to do that here: https://docs.docker.com/install/. Some tutorials on how to use Docker can be found here: https://docker-curriculum.com/.

In order to run these Docker containers on the GPU, you will also have to install nvidia-docker. nvidia-docker is an extension to Docker that lets you seamlessly hook up your docker containers to your NVIDIA GPU drivers and supporting software. You can find instructions on how to install nvidia-docker here: https://github.com/NVIDIA/nvidia-docker.

You can find documentation for the glioma segmentation module at this link: https://github.com/QTIM-Lab/DeepNeuro/tree/master/deepneuro/pipelines/Segment_GBM. Don't worry about reading it too much right now -- we will go over the parameters in this tutorial too.

Our first step will be to pull the Docker container for glioma segmentation with DeepNeuro.

In [3]:
!docker pull qtimlab/deepneuro_segment_gbm

Using default tag: latest
latest: Pulling from qtimlab/deepneuro_segment_gbm
Digest: sha256:890f03b339354f3a35aadf98045773f23d13b15ddf11df5680c7d3d68247cb3f
Status: Image is up to date for qtimlab/deepneuro_segment_gbm:latest


Easy enough! We will also have to download our medical imaging data. If you haven't already downloaded it in a previous tutorial, you can get download it from DeepNeuro with the following command. We will be loading a DICOM dataset this time, instead of a NIfTI dataset, to show off some of the preprocessing steps that come prepackaged in DeepNeuro Docker containers.

In [1]:
from deepneuro.load.load import load

training_data_sources = {
    'directories': {
                './Sample_Data/GBM_NIFTI/TRAINING':
                {'input_data': ['*_flair.nii.gz*', '*_t2.nii.gz*', '*_t1.nii.gz', '*_t1Gd.nii.gz'], 
                 'ground_truth': ['*GlistrBoost_ManuallyCorrected.nii.gz']}},
}

'./Sample_Data/TCGA_GBM_DICOM.zip'

In the folder you just downloaded, you should see four subfolders corresponding to T2, FLAIR, pre-contrast T1, and post-contrast T1 MR sequences. These images are not the same resolution, are not registered to the same patient space, are not on the same intensity scale, and have not otherwise been preprocessed. Luckily, we have a Docker container that can do all of those things, and even segment glioma tissue at the end of the process!

There are two principle ways to run a Docker container in DeepNeuro. One is directly via the Docker interface: simply construct a command and enter it into the command-line. The other is via DeepNeuro's Docker wrapper, written in Python. The latter takes care of some of the difficulties of working with Docker, and let's you run Docker containers from within a Python script, but requires you to have a local installation of DeepNeuro.

## Running a Module via Docker and the Command Line

If you are familiar with Docker containers, running DeepNeuro via the Docker interface may seem like the easiest option to you. Docker (and nvidia-docker) has its own interface for structuring command-line requests that we will follow in this portion of the tutorial.

Running containers with Docker and nvidia-docker can be easy. You begin all commands with "nvidia-docker run".

_Note that command line functions in Jupyter/Colab need to be preceded by a exclamation point. You will not need to do this on your own command line._

In [ ]:
!nvidia-docker run

So good so far. The next step is to add the "--rm" parameter. This command makes sure that upon completion of a container's task (in this case, deep learning inference), that Docker is shut down.

In [ ]:
!nvidia-docker run --rm

The next step is to specify the container you want to run. In this tutorial, we will be running the deepneuro_segment_gbm container. You access this container by pulling it from our lab's repository, "qtimlab". Thus our repository name in this command will be "qtimlab/deepneuro_segment_gbm".

In [ ]:
!nvidia-docker run --rm qtimlab/deepneuro_segment_gbm

That's all it takes to get a Docker container running! But not all that it takes to get it running and _doing something you want to do_. We'll need some more steps for that part. Particularly, we will need to submit additional commands to the Docker container to specify the action it should take. This is specified in this module's documentation, but the command you should add on is "segment_gbm pipeline". This means that DeepNeuro will run through the entire data preprocessing and segmentation pipelines for this module.

In [ ]:
!nvidia-docker run --rm qtimlab/deepneuro_segment_gbm segment_gbm pipeline

The last step is often the most difficult part for those just getting started with Docker. We have the command, but we haven't specified what data we want to run our container on it yet.

Docker requires you to mount directories on your operating system to folders inside the Docker container. Otherwise, your Docker container would not be able to access any of your files. DeepNeuro requires that you mount all data for processing into a folder titled "/INPUT_DATA". To add this to mounted directory to your docker command, you append the following parameter to your nvidia-docker call "-v [your_input_directory]:/INPUT_DATA".

Unfortunately, knowing which input directory to mount isn't obvious, so let's work through an example. Say that you downloaded your sample data in the first step the folder "my_project" shown below, and wanted to run your pipeline on the case "TCGA-02-0009".

`/home/your_username/my_project/TCGA-02-0006`

This folder has four DICOM sequences and a DICOM-SEG file. For the glioblastoma segmentation pipeline, we only need the pre- and post-contrast T1 sequences, and the FLAIR sequence. These folders have the following names:

* `/home/your_username/my_project/TCGA-02-0006/5-AX T1-62419`
* `/home/your_username/my_project/TCGA-02-0006/6-AX T1 POST-26504`
* `/home/your_username/my_project/TCGA-02-0006/4-AX FLAIR-23922`

Let's say in this example that you wanted to output your segmentations to a new folder located at this filepath:

`/home/your_username/results/TCGA-02-0006/Segmentation`

So which directory should you mount? The answer is "/home/your_username/". This is the mounted directory that contains all of the folders you need to process data with DeepNeuro. So, the command will be formatted this way:

In [ ]:
!nvidia-docker run --rm -v /home/your_username:/INPUT_DATA qtimlab/deepneuro_segment_gbm segment_gbm pipeline

All that's left is to fill in the parameters for your segmentation command accordingly. These parameters should be listed [on the documentation page for this module](https://github.com/QTIM-Lab/DeepNeuro/tree/master/deepneuro/pipelines/Segment_GBM). Note that any filepath you use will be relative to `/INPUT_DATA` rather than whatever the filepath is on your system.

Also note that because our DICOM folders have spaces in them, we will need to put our input filepaths in quotes. Otherwise, the command line will not read text on both sides of the space as a single parameter.

In [ ]:
!nvidia-docker run --rm -v /home/your_username:/INPUT_DATA qtimlab/deepneuro_segment_gbm segment_gbm pipeline \
    -T1 "/INPUT_DATA/my_project/TCGA-02-0009/5-AX T1-62419" \
    -T1POST "/INPUT_DATA/my_project/TCGA-02-0009/6-AX T1 POST-26504" \
    -FLAIR "/INPUT_DATA/my_project/TCGA-02-0009/4-AX FLAIR-23922" \
    -output_folder /INPUT_DATA/results/TCGA-02-0009/Segmentation

And that's the full command! There are, of course, other parameters you can add on to the end of the command on [the documentation page](https://github.com/QTIM-Lab/DeepNeuro/tree/master/deepneuro/pipelines/Segment_GBM) if you wish you customize your command further.

## Running DeepNeuro Dockers with a Python Wrapper

There are a lot of easy mistakes to make when creating a command to enter on the console as above. You could not realize your data has spaces in it, and forget to surround your entry in quotes. You could forget a parameter, or make some small semantic error when calling your Docker container. You could get the directory mounting wrong. Furthermore, you might want to do processing on many input cases with a DeepNeuro Docker, but you do not have the bash scripting knowledge to do it straight from the console.

Luckily, there's a better way! By install the DeepNeuro Python library, you can have access to a convenient Python function for each DeepNeuro module. These functions will create nvidia-docker commands for you, but with ordinary Python syntax that you can easily slot into your pipelines.

To try one out, the first thing we need to do is to install DeepNeuro using Python's package manager, pip:

In [ ]:
!pip install deepneuro

Hopefully, that was easy enough! We are now going to try and run the same command we ran in the previous section using DeepNeuro's python wrapper. The first step is to import the `nvidia_docker_wrapper` function.

In [ ]:
from deepneuro.docker.docker_cli import nvidia_docker_wrapper